# 6. Memória: referências, mutabilidade, e coleta de lixo

Este notebook contém exemplos do livro [_Fluent Python, Second Edition_](https://www.fluentpython.com/) .

## Variáveis não são caixas

Alguns professores sugerem que "variáveis são como caixas".
Essa analogia faz sentido em linguagens como C ou Pascal,
mas não funciona em Python, Java, JS, Ruby,
ou qualquer outra linguagem onde variáveis são referências a objetos.


Veja um código que quebra a ideia de que variáveis são caixas:

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

Se `a` e `b` são como "caixas", como explicar o que aconteceu? Mas se você imaginar que `a` e `b` são duas etiquetas coladas no mesmo objeto, faz sentido.

![Fig. 6.1](https://raw.githubusercontent.com/fluentpython/images/master/object-refs/var-boxes-x-labels.png)



## Objetos são criados antes de variáveis

O código abaixo prova que o lado direito da instrução `x = Gizmo()` é executado primeiro,
e só depois acontece a atribuição.

Primeiro a classe `Gizmo`, que apenas exibe uma mensagem quando uma instância é criada.

In [2]:
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self):#x}')
        
x = Gizmo()

Gizmo id: 0x104f60b10


In [ ]:
x

In [ ]:
'x' in globals()

In [ ]:
# y = Gizmo() * Gizmo()  # isso vai gerar um erro

In [ ]:
'y' in globals()

In [ ]:
# y  # isso vai gerar outro erro

## Identidade, igualdade, e aliases

> **NOTA:** A palavra inglesa *alias* pode ser traduzida como *apelido* ou *alcunha*.

Pelé era o apelido do jogador Edson Arantes do Nascimento.
Pelé não é apenas igual a Edson, os dois nomes se referem à mesma pessoa.

Veja esta ideia em Python:

In [ ]:
edson = dict(nome='Edson Arantes do Nascimento', ano=1940)
edson

In [ ]:
pelé = edson
pelé is edson

In [ ]:
id(pelé), id(edson)

In [ ]:
pelé['gols'] = 1283
edson

Agora imagine um impostor, tentando se passar por Pelé.
O impostor alega que tem os mesmos dados:

In [ ]:
impostor = {'nome': 'Edson Arantes do Nascimento', 'ano': 1940, 'gols': 1283}

In [ ]:
impostor is pelé

In [ ]:
impostor == pelé

In [ ]:
impostor is not pelé

## Como escolher entre `==` ou `is`

É muito mais comum a gente comparar o *valor* de dois objetos do que a *identidade* deles.

Para comparar valores, sempre use `==`. Quase sempre é o que você quer.

O caso mais comum de uso de `is` é comparar com objetos únicos (singletons),
geralmente usados como sinalizadores ou sentinelas.

Quando um objeto é único é mais eficiente verificar a identidade do que a igualdade,
porque o operador `==` pode ser sobrecarregado implementando o método `__eq__`,
portanto toda vez que aparece `==`, o interpretador precisa verificar a presença do método `__eq__`.

Mas o operador `is` não pode ser sobrecarregado.
Ele é implementado em C simplesmente comparando o *id* dos objetos.

Exemplo:

In [ ]:
from unicodedata import name

for código in range(0x30):
    car = chr(código)
    nome = name(car, None)
    if nome is None:
        continue
    print(f'U+{código:04x}\t{car}\t{nome}')

Sentinelas são valores especiais usados em filas ou sockets
para sinalizar o fim de uma sequência de valores.

Exemplo:

In [ ]:
from queue import SimpleQueue

FIM_DA_SÉRIE = object()
BICICLETA = object()

FIM_DA_SÉRIE == BICICLETA

In [ ]:
FIM_DA_SÉRIE is not BICICLETA

In [ ]:
def preparar():
    fila = SimpleQueue()
    for letra in 'ABC':
        fila.put(letra)
    fila.put(FIM_DA_SÉRIE)
    return fila

fila = preparar()
    
while (item := fila.get()) is not FIM_DA_SÉRIE:
    print(item)  

> **NOTA:** Porque não usar um laço `for` para percorrer uma fila?<br>Porque a operação `fila.get()` bloqueia quando não há itens na fila. Filas são muito usadas para sincronizar e trocar dados entre threads.

----

### 🤔 Operador Morsa `:=`

O laço **`while`** acima usa o "operador morsa" `:=` introduzido no Python 3.8.

O exemplo acima antes do Python 3.8 seria escrito assim:

In [ ]:
fila = preparar()

while True:
    item = fila.get()
    if item is FIM_DA_SÉRIE:
        break
    print(item)

Chama-se "operador morsa" por causa do emoticon `:=` que representa uma morsa:

<img src="https://upload.wikimedia.org/wikipedia/commons/c/ce/Noaa-walrus22.jpg" width="300">

----

## Afinal, tuplas são mesmo imutáveis?

A melhor resposta é: **depende**.

Uma tupla é um _container_: uma estrutura de dados que contém referências a outros objetos.
Outros containers são listas, dicts, sets, e filas.

> Em contraste, `str`, `bytes` e `array` são _flat coleções planas_: armazenam diretamente seus dados, e não referências a outros objetos.

Em uma tupla, as referências são imutáveis.
Ou seja, uma vez criada a tupla, ela sempre apontará para a mesma sequência de objetos.

Porém, se qualquer objeto contido na tupla for mutável,
e seu valor for alterado, isso mudará também o valor da tupla!

In [ ]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
t1 == t2  # valores iguais

In [ ]:
t1 is t2  # identidades diferentes

In [ ]:
id(t1[-1])  # repare no id do objeto lista [30, 40]

In [ ]:
t1[-1].append(99)  # alterando a lista t1[-1]
t1

In [ ]:
t2  # nada muda em t2

In [ ]:
t1 == t2  # agora as tuplas tem valores diferentes

In [ ]:
id(t1[-1])  # mas o id de t1[-1] não mudou

👉 **RESUMO:** o que nunca muda em uma tupla são as referências que ela contém. Mas se o valor de um item da tupla mudar, isso muda o valor da tupla.

Em outras palavras: a identidade dos itens em uma tupla nunca muda, mas seus valores podem mudar, dependendo do tipo do item.

Copiar um conteiner abre uma questão: os itens do novo container devem ser cópias dos itens originais, ou podem ser referências para os mesmos itens? Esse é o tema da próxima sessão.